In [155]:
import requests   # best library to manage HTTP transactions
from bs4 import BeautifulSoup # web-scraping library
import json
from time import sleep
import csv
import math
from fuzzywuzzy import fuzz # fuzzy logic matching
from fuzzywuzzy import process
import xml.etree.ElementTree as et # library to traverse XML tree
import urllib
import datetime
import string
from pathlib import Path
import copy # import the copy module from the standard library

import vb_common_code as vbc

employerQId = 'Q29052' # Vanderbilt University
sparqlSleep = 0.1 # number of seconds to wait between queries to SPARQL endpoint
testRatio = 90 # similarity required for a potential match of a generic wikidata match
journalTestRatio = 94 # similarity required for a potential match of a generic wikidata match

home = str(Path.home()) # gets path to home directory; supposed to work for both Win and Mac
csv_path = home + '/divinity-law/vulibDivinity.csv'
known_name_strings_path = home + '/divinity-law/known-name-strings.csv'
wikidata_article_matches_path  = home + '/divinity-law/wikidata-article-matches.csv'
duplicate_works_path  = home + '/divinity-law/duplicate.csv'
journals_path = home + '/divinity-law/journals.csv'

# read from a CSV file beginning with a bit order mark (BOM) into a list of dictionaries
# Change encoding type to utf-8-sig instead of utf-8 to fix this
def read_dict_bom(filename):
    with open(filename, 'r', newline='', encoding='utf-8-sig') as fileObject: 
        dictObject = csv.DictReader(fileObject)
        array = []
        for row in dictObject:
            array.append(row)
    return array

def reverse_names(nameLastFirst):
    nameLastFirst = nameLastFirst.strip()
    nameParts = nameLastFirst.split(',')
    if len(nameParts) < 2: # name probably isn't reversed
        name = nameLastFirst.strip()
    else:
        firstName = nameParts[1].strip()
        lastName = nameParts[0].strip()
        name = firstName + ' ' + lastName

        if len(nameParts) > 2:
            suffix = nameParts[2].strip()
        else:
            suffix = ''
        name = firstName + ' ' + lastName
        if suffix == 'Jr.':
            name += ', Jr.'
        elif suffix == 'Jr':
            name += ', Jr.'
        elif suffix == '':
            pass
        elif suffix == 'II':
            name += ' ' + suffix
        elif suffix == 'III':
            name += ' ' + suffix
        elif suffix == 'IV':
            name += ' ' + suffix
        elif suffix == 'V':
            name += ' ' + suffix
    return name

def generateHeaderDictionary(acceptMediaType):
    userAgentHeader = 'VanderBot/1.0 (https://github.com/HeardLibrary/linked-data/tree/master/publications; mailto:steve.baskauf@vanderbilt.edu)'
    requestHeaderDictionary = {
        'Accept' : acceptMediaType,
        'User-Agent': userAgentHeader
    }
    return requestHeaderDictionary

# extracts the qNumber from a Wikidata IRI
def extract_qnumber(iri):
    # pattern is http://www.wikidata.org/entity/Q6386232
    pieces = iri.split('/')
    return pieces[4]



In [156]:
# read in data from Zotero download CSV
publications = read_dict_bom(csv_path) # need to use special function since Zotero output has BOM
print(len(publications),'publications read from CSV')

# read in previously discovered Q IDs and names
known_name_strings = vbc.readDict(known_name_strings_path)
print(len(known_name_strings),'discovered authors Q IDs read from CSV')

# read in previously discovered matches of articles to Wikidata
wikidata_article_matches = vbc.readDict(wikidata_article_matches_path)
print(len(wikidata_article_matches),'previously discovered article matches read from CSV')

# read in list of works determined to be duplicates
duplicate_works = vbc.readDict(duplicate_works_path)
print(len(duplicate_works),'duplicate works list items read from CSV')

# read in list of all Q IDs and names of journals in Wikidata
journals_in_wikidata = vbc.readDict(journals_path)
print(len(journals_in_wikidata),'journals read from CSV')

# Download the labels and descriptions of all existing institutional people
org_label_query = vbc.Query(labelscreen='?id wdt:P1416 ?deptOrCollege.?deptOrCollege wdt:P749+ wd:' + employerQId + '.', sleep=sparqlSleep)
org_labels = org_label_query.labels_descriptions('')
print(len(org_labels), 'Wikidata people labels downloaded')

'''
org_description_query = vbc.Query(labeltype='description', labelscreen='?id wdt:P1416 ?deptOrCollege.?deptOrCollege wdt:P749+ wd:' + employerQId + '.', sleep=sparqlSleep)
org_descriptions = org_description_query.labels_descriptions('')
print(len(org_descriptions), 'descriptions downloaded')
'''

2819 publications read from CSV
114 discovered authors Q IDs read from CSV
0 previously discovered article matches read from CSV
18100 journals read from CSV
4439 Wikidata people labels downloaded


"\norg_description_query = vbc.Query(labeltype='description', labelscreen='?id wdt:P1416 ?deptOrCollege.?deptOrCollege wdt:P749+ wd:' + employerQId + '.', sleep=sparqlSleep)\norg_descriptions = org_description_query.labels_descriptions('')\nprint(len(org_descriptions), 'descriptions downloaded')\n"

In [ ]:
output_list = []
identified_authors_list = [] # list of dictionaries
unidentified_authors_list = [] # list of strings
identified_journals_list = [] # list of dictionaries
unidentified_journals_list = [] # list of dictionaries

count = 0
for publication in publications:
    count += 1
    # print something every 100 rows
    if count % 100 == 0:
        print(count)
        
    # Set up the output row by starting with the input row
    # The input row is a dictionary
    output_row = copy.deepcopy(publication) # use deepcopy rather than a reference
        
    # *********** Author matching ***********
    
    # First separate the authors if more than one
    if ';' in publication['Author']:
        names = publication['Author'].split(';')
    else:
        names = [publication['Author']]
    names_list = []
    
    # Process each of the authors of the publication
    for name in names:
        # The name dict is where we store names and Q IDs (if discovered)
        # for all of the authors of the paper
        name_dictionary = {}
        
        # Guard against errors in the case where authors are missing
        if name.strip() != '':
            
            # Neet to revers the names if they are last name first
            name_string = reverse_names(name)
            
            # First check whether it's a name that matches a know Div School person
            found = False
            for label in org_labels:
                # NOTE: used token_set_ratio here because of the wide variety of 
                # ways that names are sliced and diced. Need to determine how 
                # often there are false positives.
                setRatio = fuzz.token_set_ratio(name_string, label['string'])
                if setRatio >= testRatio: # We get a name match
                    name_dictionary['string'] = label['string'] # Add the match to the name dict
                    name_dictionary['qid'] = label['qid'] # Add the match to the name dict
                    found = True
                    matched = False
                    
                    # Find out whether we need to add to the list of IDed authors
                    for id_name in identified_authors_list:
                        if name_dictionary['string'] == id_name['string']:
                            matched = True
                            break # already in the list, do nothing more
                    if not matched: # all the way through the loop without finding, so add to list
                        identified_authors_list.append(name_dictionary)
                        
                    break # since found, quit the loop looking for that author's label
                    
            # If the name loop wasn't broken, move on to checking the 
            # list we keep of identified people who don't match the VU list
            for label in known_name_strings:
                if name_string == label['string']: # We get a name match with dirty name string
                    name_dictionary['qid'] = label['qid'] # Add the discovered name to article name dict
                    name_dictionary['string'] = label['label'] # Add the Wikidata label for the match
                    found = True
                    matched = False
                    
                    # Find out whether we need to add this match to the list of IDed authors
                    for id_name in identified_authors_list:
                        if name_dictionary['string'] == id_name['string']:
                            matched = True
                            break
                    if not matched:
                        identified_authors_list.append(name_dictionary)
                    break
                
            # If there is no match to either name source, then don't give a Q ID
            if not found:
                #print('none', name_string)
                name_dictionary['qid'] = ''
                name_dictionary['string'] = name_string
                matched = False
                
                # Find out if this is a new unidentified name or 
                # if it's already on the list of unIDed people
                for unid_name in unidentified_authors_list:
                    if name_string == unid_name['label']:
                        matched = True
                        break # quit checking on more unIDed people
                if not matched: # need to add to the list if never found
                    unidentified_authors_list.append({'label': name_string, 'listing': name.strip()})
            names_list.append(name_dictionary)
    output_row['author_qids_names'] = names_list 
    #print(names_list)
    
    
    # *********** Journal matching ***********

    # only process rows for journal articles
    if publication['Item Type'] == 'journalArticle':
        # first see if the Publication Title is one we already know about
        found_in_id_journals = False
        for known_journal in identified_journals_list:
            setRatio = fuzz.ratio(publication['Publication Title'], known_journal['label'])
            if setRatio >= journalTestRatio: # We get a name match
                found_in_id_journals = True
                output_row['publication_title_qid'] = known_journal['qid'] # assign this Q ID for the journal of the row
                # don't need to keep looking so quit the search loop
                break
                
        # need to check the big list if the journal is new
        if not found_in_id_journals:
            found_in_poss_journals = False
            for possible_journal in journals_in_wikidata:
                # Used regular ratio rather than token_set_ratio here. There were false positives
                # Due to the many similar journals that had overlapping name parts.
                setRatio = fuzz.ratio(possible_journal['label'], publication['Publication Title'])
                if setRatio >= journalTestRatio: # We get a title string match
                    found_in_poss_journals = True
                    output_row['publication_title_qid'] = possible_journal['qid'] # assign this Q ID for the journal of the row
                    print(possible_journal['qid'], possible_journal['label'], setRatio, publication['Publication Title'])
                    identified_journals_list.append({'qid': possible_journal['qid'], 'label': possible_journal['label']})
                    break
            if not found_in_poss_journals:
                matched = False
                for unid_journal in unidentified_journals_list:
                    if publication['Publication Title'] == unid_journal:
                        matched = True
                        break
                if not matched:
                    unidentified_journals_list.append(publication['Publication Title'])
    output_list.append(output_row)

# Save the data
filename = home + '/divinity-law/output.csv'
fieldnames = ['author_qids_names', 'publication_title_qid'] + list(publications[0].keys())
vbc.writeDictsToCsv(output_list, filename, fieldnames)

# Save the identified authors
filename = home + '/divinity-law/identified-authors.csv'
fieldnames = ['qid', 'string']
vbc.writeDictsToCsv(identified_authors_list, filename, fieldnames)

# Save the identified journals
filename = home + '/divinity-law/identified-journals.csv'
fieldnames = ['qid', 'label']
vbc.writeDictsToCsv(identified_journals_list, filename, fieldnames)

# Save the list of unidentified authors
filename = home + '/divinity-law/unidentified-authors.csv'
fieldnames = ['label', 'listing']
vbc.writeDictsToCsv(unidentified_authors_list, filename, fieldnames)

# Save the list of unidentified journals
filename = home + '/divinity-law/unidentified-journals.txt'
with open(filename, 'wt', encoding='utf-8') as fileObject:
    for journal in unidentified_journals_list:
        print(journal, file=fileObject)

# Save the discovered article matches
fieldnames = ['Key', 'qid']
vbc.writeDictsToCsv(wikidata_article_matches_path, filename, fieldnames)

# Save the discovered duplicate works
fieldnames = ['Key', 'duplicatekey']
vbc.writeDictsToCsv(duplicate_works_path, filename, fieldnames)

print('done')


In [158]:
print(list(publications[0].keys()))
print(publications[0]['Key'])

['Key', 'Item Type', 'Publication Year', 'Author', 'Title', 'Publication Title', 'ISBN', 'ISSN', 'DOI', 'Url', 'Abstract Note', 'Date', 'Date Added', 'Date Modified', 'Access Date', 'Pages', 'Num Pages', 'Issue', 'Volume', 'Number Of Volumes', 'Journal Abbreviation', 'Short Title', 'Series', 'Series Number', 'Series Text', 'Series Title', 'Publisher', 'Place', 'Language', 'Rights', 'Type', 'Archive', 'Archive Location', 'Library Catalog', 'Call Number', 'Extra', 'Notes', 'File Attachments', 'Link Attachments', 'Manual Tags', 'Automatic Tags', 'Editor', 'Series Editor', 'Translator', 'Contributor', 'Attorney Agent', 'Book Author', 'Cast Member', 'Commenter', 'Composer', 'Cosponsor', 'Counsel', 'Interviewer', 'Producer', 'Recipient', 'Reviewed Author', 'Scriptwriter', 'Words By', 'Guest', 'Number', 'Edition', 'Running Time', 'Scale', 'Medium', 'Artwork Size', 'Filing Date', 'Application Number', 'Assignee', 'Issuing Authority', 'Country', 'Meeting Name', 'Conference Name', 'Court', 'Refe

In [129]:
alternatives = ''
for author in identified_authors_list:
    alternatives += 'wd:' + author['qid'] + '\n'

# create a string for the query
query = '''
select distinct ?article ?articleLabel
    where {
        VALUES ?id
    {
''' + alternatives + '''
    }
    ?article wdt:P50 ?id.
    {?article wdt:P31 wd:Q18918145.} union {?article wdt:P31 wd:Q13442814}
    ?article rdfs:label ?articleLabel.
    filter(lang(?articleLabel)='en')
}'''

#print(query)



select distinct ?article ?articleLabel
    where {
        VALUES ?id
    {
wd:Q83505898
wd:Q92216692
wd:Q83505887
wd:Q92207043
wd:Q92207288
wd:Q29447340
wd:Q27643173
wd:Q83500312
wd:Q83505894
wd:Q16122562
wd:Q83505860
wd:Q63038670
wd:Q83505836
wd:Q53901122
wd:Q4749053
wd:Q91188434
wd:Q87400652
wd:Q83422314
wd:Q7595851
wd:Q83432379
wd:Q82775133
wd:Q6525772
wd:Q20708438
wd:Q55583731
wd:Q83389821
wd:Q63430468
wd:Q92206914
wd:Q92207578
wd:Q91909582
wd:Q83500331
wd:Q52274406
wd:Q73037010
wd:Q83505891
wd:Q4772041
wd:Q83505877
wd:Q83505983
wd:Q63038665
wd:Q6113796
wd:Q86366619
wd:Q92207880
wd:Q67554073
wd:Q83492880
wd:Q71850074
wd:Q83505989
wd:Q83505844
wd:Q84486
wd:Q16973499
wd:Q16830980
wd:Q83505863
wd:Q83500320
wd:Q86835641
wd:Q92224966
wd:Q83260986
wd:Q83505820
wd:Q92212610
wd:Q92212616
wd:Q15379207
wd:Q63183857
wd:Q83505948
wd:Q92213987
wd:Q5218399
wd:Q30302655
wd:Q19595159
wd:Q1328650
wd:Q6200439
wd:Q5473986
wd:Q18390705
wd:Q630122
wd:Q194754
wd:Q25455836
wd:Q89052941
wd:Q6303611
wd:Q

In [136]:
existing_articles = []
wikidataEndpointUrl = 'https://query.wikidata.org/sparql'
acceptMediaType = 'application/json'
r = requests.get(wikidataEndpointUrl, params={'query' : query}, headers = generateHeaderDictionary(acceptMediaType))
try:
    data = r.json()
    statements = data['results']['bindings']
    for statement in statements:
        wikidataIri = statement['article']['value']
        article_qnumber = extract_qnumber(wikidataIri)
        article_label = statement['articleLabel']['value']
        existing_articles.append({'qid': article_qnumber, 'label': article_label})
except:
    existing_articles = [r.text]
# delay a quarter second to avoid hitting the SPARQL endpoint to rapidly
sleep(sparqlSleep)

print(json.dumps(existing_articles, indent=2))


[
  {
    "qid": "Q53141308",
    "label": "Revisiting the living human web: theological education and the role of clinical pastoral education."
  },
  {
    "qid": "Q69685319",
    "label": "Marital therapy caught between person and public: Christian traditions on marriage"
  },
  {
    "qid": "Q69685326",
    "label": "Also a pastoral theologian: in pursuit of dynamic theology (or: meditations from a recalcitrant heart)"
  },
  {
    "qid": "Q69685321",
    "label": "Teaching practical theology: introducing six perspectives"
  },
  {
    "qid": "Q69685324",
    "label": "Embodied knowing, embodied theology: what happened to the body?"
  },
  {
    "qid": "Q69685325",
    "label": "Five misunderstandings about practical theology"
  },
  {
    "qid": "Q69685331",
    "label": "Cognitive neuroscience and the question of theological method"
  },
  {
    "qid": "Q69685330",
    "label": "Words made flesh: writings in pastoral and practical theology"
  },
  {
    "qid": "Q69685333",
    "l

In [159]:

count = 0
for publication in publications:
    count += 1
    # print something every 100 rows
    if count % 100 == 0:
        print(count)
        
    # only process rows for journal articles
    if publication['Item Type'] == 'journalArticle':
        for article in existing_articles:
            #setRatio = fuzz.token_set_ratio(article['label'], publication['Title'])
            setRatio = fuzz.ratio(article['label'].lower(), publication['Title'].lower())
            #if setRatio >= 95: # We get a name match
            if setRatio >= 80: # We get a name match
                if article['label'] != publication['Title']:
                    print('Match ratio:', setRatio)
                    print(article['label'])
                    print('https://www.wikidata.org/wiki/' + article['qid'])
                    print('row:', count + 1)
                    print(publication['Title'])
                    print(publication['Author'])
                    print()
print('done')

100
Match ratio: 92
Eros for the other: retaining truth in a pluralistic world
https://www.wikidata.org/wiki/Q69685414
row: 149
Review of Eros for the other: retaining truth in a pluralistic world
Armour, Ellen T.

200
300
400
500
600
700
800
900
Match ratio: 100
The wrestle of Christ and culture in pragmatic public theology
https://www.wikidata.org/wiki/Q92212019
row: 923
The Wrestle of Christ and Culture in Pragmatic Public Theology
Anderson, Victor

Match ratio: 100
The narrative turn in Christian ethics : a critical appraisal
https://www.wikidata.org/wiki/Q92214070
row: 925
The Narrative Turn in Christian Ethics : A Critical Appraisal
Anderson, Victor

Match ratio: 100
Black scholarly aesthetics and the religious critic : black experience as manifolds of manifestations and powers of presentations
https://www.wikidata.org/wiki/Q91649499
row: 945
Black scholarly aesthetics and the religious critic: black experience as manifolds of manifestations and powers of presentations
Anderson, 

In [236]:
def retrieveCrossRefDoi(doi):
    metadata = {}
    authorList = []
    crossRefEndpointUrl = 'https://api.crossref.org/works/'
    encodedDoi = urllib.parse.quote(doi)
    searchUrl = crossRefEndpointUrl + encodedDoi
    acceptMediaType = 'application/json'
    response = requests.get(searchUrl, headers=vbc.generateHeaderDictionary(acceptMediaType))
    if response.status_code == 404:
        authorList = [] # return an empty list if the DOI won't dereference at CrossRef
    else:
        if 1==1:
        #try:
            data = response.json()
            message = data['message']
            #print(json.dumps(data, indent = 2))
            if 'author' in message:
                authors = message['author']
                for author in authors:
                    authorDict = {}
                    if 'ORCID' in author:
                        authorDict['orcid'] = author['ORCID']
                    else:
                        authorDict['orcid'] = ''
                    if 'given' in author:
                        authorDict['givenName'] = author['given']
                    else:
                        authorDict['givenName'] = ''
                    if 'family' in author:
                        authorDict['familyName'] = author['family']
                    else:
                        authorDict['familyName'] = ''
                    affiliationList = []
                    if 'affiliation' in author:
                        for affiliation in author['affiliation']:
                            affiliationList.append(affiliation['name'])
                    # if there aren't any affiliations, the list will remain empty
                    authorDict['affiliation'] = affiliationList
                    authorList.append(authorDict)
            metadata['authors'] = authorList
            if 'issue' in message:
                metadata['issue'] = message['issue']
            else:
                metadata['issue'] = ''
                
            if 'page' in message:
                metadata['page'] = message['page']
            else:
                metadata['page'] = ''
            
            if 'volume' in message:
                metadata['volume'] = message['volume']
            else:
                metadata['volume'] = ''
            
            if 'language' in message:
                metadata['language'] = message['language']
            else:
                metadata['language'] = ''
            
            if 'issued' in message:
                metadata['date'] = message['issued']['date-parts'][0][0]
            else:
                metadata['date'] = ''
            
            if 'title' in message:
                metadata['title'] = message['title'][0]
            else:
                metadata['title'] = ''
            
            if 'container-title' in message:
                metadata['journaltitle'] = message['container-title'][0]
            else:
                metadata['journaltitle'] = ''
            
            if 'issn-type' in message:
                print_flag = False
                electronic_flag = False
                for issntype in message['issn-type']:
                    if issntype['type'] == 'print':
                        print_flag = True
                        metadata['printissn'] = issntype['value']
                    if issntype['type'] == 'electronic':
                        electronic_flag = True
                        metadata['electronicissn'] = issntype['value']
                if not print_flag:
                    metadata['printissn'] = ''
                if not electronic_flag:
                    metadata['electronicissn'] = ''
            else:
                metadata['printissn'] = ''
                metadata['electronicissn'] = ''
            
       #except:
        #    metadata = data
    return metadata


In [237]:
data = retrieveCrossRefDoi('10.5325/jafrireli.3.1.0044')
print(json.dumps(data, indent=2))

{
  "authors": [
    {
      "orcid": "",
      "givenName": "",
      "familyName": "Floyd-Thomas",
      "affiliation": []
    }
  ],
  "issue": "1",
  "page": "44",
  "volume": "3",
  "language": "",
  "date": 2015,
  "title": "Gaining One's Definition",
  "journaltitle": "Journal of Africana Religions",
  "printissn": "2165-5405",
  "electronicissn": ""
}


In [223]:
def search_cross_ref(query):
    metadata = {}
    #crossRefEndpointUrl = 'https://api.crossref.org/journal-article/works?'
    crossRefEndpointUrl = 'https://api.crossref.org/works?'
    #encodedDoi = urllib.parse.quote(doi)
    searchUrl = crossRefEndpointUrl + query
    acceptMediaType = 'application/json'
    response = requests.get(searchUrl, headers=vbc.generateHeaderDictionary(acceptMediaType))
    print(response.url)
    if response.status_code == 404:
        metadata = {} # return an empty list if the DOI won't dereference at CrossRef
    else:
        metadata = response.json()
    return metadata

In [234]:
query = 'query.author=Amy-Jill'
data = search_cross_ref(query)
print(json.dumps(data, indent=2))

https://api.crossref.org/works?query.author=Amy-Jill
{
  "status": "ok",
  "message-type": "work-list",
  "message-version": "1.0.0",
  "message": {
    "facets": {},
    "total-results": 203411,
    "items": [
      {
        "indexed": {
          "date-parts": [
            [
              2020,
              4,
              7
            ]
          ],
          "date-time": "2020-04-07T07:08:24Z",
          "timestamp": 1586243304715
        },
        "reference-count": 3,
        "publisher": "Informa UK Limited",
        "issue": "2",
        "content-domain": {
          "domain": [],
          "crossmark-restriction": false
        },
        "short-container-title": [
          "Religion"
        ],
        "published-print": {
          "date-parts": [
            [
              2002,
              4
            ]
          ]
        },
        "DOI": "10.1006/reli.2002.0419",
        "type": "journal-article",
        "created": {
          "date-parts": [
            [


In [235]:
print(json.dumps(data['message']['items'][19], indent=2))

{
  "indexed": {
    "date-parts": [
      [
        2020,
        3,
        30
      ]
    ],
    "date-time": "2020-03-30T05:14:02Z",
    "timestamp": 1585545242528
  },
  "reference-count": 34,
  "publisher": "Informa UK Limited",
  "issue": "2",
  "funder": [
    {
      "DOI": "10.13039/501100000882",
      "name": "University of Aberdeen",
      "doi-asserted-by": "publisher",
      "award": []
    }
  ],
  "content-domain": {
    "domain": [
      "www.tandfonline.com"
    ],
    "crossmark-restriction": true
  },
  "short-container-title": [
    "Journal of Agromedicine"
  ],
  "published-print": {
    "date-parts": [
      [
        2018,
        4,
        3
      ]
    ]
  },
  "DOI": "10.1080/1059924x.2017.1423000",
  "type": "journal-article",
  "created": {
    "date-parts": [
      [
        2018,
        4,
        12
      ]
    ],
    "date-time": "2018-04-12T17:18:51Z",
    "timestamp": 1523553531000
  },
  "page": "154-165",
  "update-policy": "http://dx.doi.org/10